In [ ]:
%pip install ipywidgets

In [ ]:
import pyaudio

p = pyaudio.PyAudio(paApi=pyaudio.paALSA)

for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()

In [5]:
import pyaudio

from recase import Config, WordpieceTokenizer, Model, init, gen_pred

In [17]:
import ipywidgets as widgets
import IPython.display as display
from threading import Thread
from queue import Queue

messages = Queue()
recordings = Queue()

record_button = widgets.Button(
    description="Record",
    disabled=False,
    button_style="success",
    icon="microphone"
)

stop_button = widgets.Button(
    description="Stop",
    disabled=False,
    button_style="warning",
    icon="stop"
)

output = widgets.Output()

# Add custom CSS to allow overflow
output.add_class("output-overflow")

def start_recording(data):
    messages.put(True)
    with output:
        # display.display("Starting...")
        print("Starting...")
        record = Thread(target=record_microphone)
        record.start()

        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()

def stop_recording(data):
    with output:
        messages.get()
        # display.display("Stopped...")
        print("Stopped...")

record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display.display(record_button, stop_button, output)

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output(_dom_classes=('output-overflow',))

ALSA lib pcm_dsnoop.c:601:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib confmisc.c:1369:(snd_func_refer) Unable to find definition 'cards.1.pcm.modem.0:CARD=1'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.phoneline:CARD=1,DEV=0
ALSA lib confmisc.c:1369:(snd_func_refer) Unable to find definition 'cards.1.pcm.modem.0:CARD=1'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.phoneline:CARD=1,DEV=0
ALSA lib confmisc.c:1369:(snd_func_refer) Unable to find definition 'cards.1.pcm.modem.0:CARD=1'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned

In [14]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 3
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 0

def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()
    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate=FRAME_RATE,
                    input=True,
                    frames_per_buffer=chunk
                    )
    frames = []
    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)

        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames=[]

    stream.stop_stream()
    stream.close()
    p.terminate()

In [13]:
import subprocess
import json
from vosk import Model, KaldiRecognizer
import time

model = Model(model_name="vosk-model-en-us-0.22")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

def speech_recognition(output):
    temp_text = ''
    while not messages.empty():
        frames = recordings.get()
        waveform_data = b''.join(frames)
        rec.AcceptWaveform(waveform_data)
        result = rec.Result()

        text1 = json.loads(result)["text"]
        # load_text = temp_text + text1


        # config = Config(action='predict', lang='en', flavor=None, device='cpu')
        # init(config)
        # checkpoint_path = 'vosk-recasepunc-en-0.22/checkpoint'

        # cased = gen_pred(config, checkpoint_path, load_text)

        # puncs = ['.', '?', '!']

        # last_punctuation_index = max([cased.rfind(p) for p in puncs])

        # if cased[-1] in puncs:
        #     pass
        
        # elif last_punctuation_index == -1:
        #     temp_text = cased
        #     cased = ''
        # else:
        #     temp_text = cased[last_punctuation_index + 1:].strip()
        #     cased = cased[:last_punctuation_index + 1].strip()

        # cased = subprocess.check_output("python vosk-recasepunc-en-0.22/recasepunc.py predict vosk-recasepunc-en-0.22/checkpoint", shell=True, text=True, input=text1)
        output.append_stdout(text1)
        # with output:
        #     output.append_stdout(cased)
        #     print(cased)
        time.sleep(1)


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:11:12:13:14:15
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /home/saadsameerkhan/.cache/vosk/vosk-model-en-us-0.22/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:279) Loading HCLG from /home/saadsameerkhan/.cache/vosk/vosk-model-en-us-0.22/graph/HCLG.fst
LOG (VoskAPI:ReadDataFiles():model.cc:297) Loading words from /home/saadsameerkhan/.cache/vosk/vosk-model-en-us-0.22/graph/words.txt
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo /home/saadsame

In [ ]:
t1='hello this is saad and today we are going to be talking about the fundamentals of programming subscribing really helps the channel so please make sure to leave a like how about we go over the details now'
config = Config(action='predict', lang='en', flavor=None, device='cpu')
init(config)
checkpoint_path = 'vosk-recasepunc-en-0.22/checkpoint'

gen_pred(config, checkpoint_path, t1)